In [1]:
import requests
import threading
import pandas as pd
import flask
from flask import Flask, request
import geopy
from geopy import Nominatim
from translate import Translator
from langdetect import detect
from iso639 import languages
from textblob import TextBlob

In [2]:
key = 'fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w='

# this method helps us get long and lat of certain city

def get_latlong(b):

    
    try:
    
        city = b

        geolocator = Nominatim(user_agent = 'Safra')
    
        loc = geolocator.geocode(city)
         
        # by default
    except: return 'No results found' , f'{b}'
    
    return loc.latitude , loc.longitude

In [3]:
# this is a test
lat , long = get_latlong('jeddah')

In [4]:
lat , long

(21.5810088, 39.1653612)

In [5]:
def translate(array):
        
    # this list contains new translated descriptions
    tlds = []
    
    for desc in array:
        try:
            # Specifying the language for
            # detection
            # dbt : detection before translation
            dbt = detect(desc)

            # saving desc into text to translate
            if desc != 'Not Available':
                text = desc

                blob = TextBlob(text)

                # tat : text after translation
                tat = blob.translate(from_lang = detect(desc) , to = 'en')
                 # if description is already in english ---> save original description
                if dbt != 'en':
                    tlds.append(str(tat))

            # if not --> save translated description
            else:
                tlds.append(desc)
        except: tlds.append(desc)
        
    return tlds

In [6]:
def extract_categories(array):
    templist = [array[i][0]['name'] for i in range(len(array))]
    
    return templist
    

In [7]:
def add_photos(array):
    
    return

In [8]:
def retrieve_places(a , c):

    """
    a : condition --- >  example : coffee , art gallery , etc ...
    c : city name
    """
    
    
    lat , long = get_latlong(c)
    if type(lat) != str:

        if a != '':
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&query={a}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories%2Cphotos"

        else:
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories%2Cphotos"


        url = fields_url

        headers = {
            "Accept": "application/json",
            "Authorization": key
        }

        response = requests.get(url, headers=headers)

        data = response.json()

        df = pd.json_normalize(data['results'])

        #deleting unnecessary columns

        try:
            df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'
                                           ,'location.country', 'location.region'
                                           , 'description' , 'categories']),1,inplace=True)

        except: df = df
            
        
        
        """
        in this phase we add empty columns if columns are not available already
        
        ============== remember to change range when changing number of retrieved rows ==============
        """
        if 'price' not in df.columns:
            df.insert(len(df.columns) , 'price' , ['Not Available' for i in range(10)] )
        
        if 'description' not in df.columns:
            df.insert(len(df.columns) , 'description' , ['Not Available' for i in range(10)])
        
        if 'rating' not in df.columns:
            df.insert(len(df.columns) , 'rating' , ['Not Available' for i in range(10)])

        
#         # renaming columns
        if 'location.country' in df.columns and 'location.region' in df.columns:
            df.rename(columns = {'location.country':'country' , 'location.region':'region'}, inplace = True)
    
        


        # filling nan values

        df = df.fillna('Not Available')


        # translating process starts here
        # error handling
        if 'description' in df.columns:

            # extracting
            array = df['description'].to_list()

            # tdl : translated descriptions list

            """
            in this line we call function to translate all descriptions as following

            other than english ---> translate

            Not Available ---> keep it as it is

            english description ---> keep it as it's
            """ 
            
            tdl = translate(array)

            # insert it into last 
            df.insert(df.columns.get_loc('description')+1  , 'translated_description' , tdl)

        
        
        if 'region' in df.columns:
            array_r = df['region'].to_list()
            
            trl = translate(array_r)
            
            df.insert(df.columns.get_loc('region')+1 , 'translated_region' , trl)

        if 'categories' in df.columns:
            templist = df['categories'].to_list()
            templist = extract_categories(templist)
            
            df.drop(['categories'] , inplace = True , axis = 1)
            df.insert(len(df) , 'categories' , templist)
            
        try:
            # changing datatypes
            df = df.astype({'price': str , 'rating': str})
        except: df = df


        data = df.to_json(orient = 'records')
        return df , data
    else:
        return lat , long 
    

In [9]:
'''

test field


'''

'\n\ntest field\n\n\n'

In [10]:
df, data_json = retrieve_places('coffee' , 'riyadh')
df

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_34628\4038082940.py:35: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'


,fsq_id,description,translated_description,name,price,rating,tel,country,region,translated_region,categories
0,5d27582cb05fbc0023959185,The Starbucks Experience is more than a cup of...,The Starbucks Experience is more than a cup of...,ستاربكس,1,9.2,011 520 0517,SA,Riyadh Province,Riyadh Province,Coffee Shop
1,566db885498e8b884eecff76,The Starbucks Experience is more than a cup of...,The Starbucks Experience is more than a cup of...,Starbucks (ستاربكس),2,9.4,011 511 1838,SA,Riyadh Province,Riyadh Province,Coffee Shop
2,5dff076de60b0b0008c1000a,Not Available,Not Available,ستاربكس,1,9.2,Not Available,SA,Riyadh Province,Riyadh Province,Coffee Shop
3,5ee0d21dcc02ad45c5da07ad,Not Available,Not Available,Dear You,1,9.1,Not Available,SA,Riyadh Province,Riyadh Province,Coffee Shop
4,539ec8c5498e78925a461270,شركة كلمات... لخدمات الأطعمة والحلويات هي شركة...,Words Company ... Food and Sweets Services is ...,كلمات للحلويات,1,9.3,011 812 0436,SA,Riyadh Province,Riyadh Province,Donut Shop
5,5dc3325ae167fe00084f61ce,Not Available,Not Available,Elixir Bunn Coffee Roasters (محمصة و مقهى إكسي...,1,9.3,9200 13926,SA,Riyadh Province,Riyadh Province,Coffee Shop
6,5da04eee201e260007102a63,Not Available,Not Available,Starbucks (ستاربكس),1,9.4,Not Available,SA,Riyadh Province,Riyadh Province,Coffee Shop
7,5633543e498efb57821f71ef,Not Available,Not Available,Starbucks (ستاربكس),2,9.1,011 488 9301,SA,Riyadh Province,Riyadh Province,Coffee Shop
8,58ffa898610f041c1cd1edfb,Not Available,Not Available,4Twins Coffee & Sweet,1,9.2,053 330 6923,SA,Riyadh Province,Riyadh Province,Coffee Shop
9,5bd484aae97dfb002c827a42,Not Available,Not Available,IDMI Coffee Roasting Co. (محمصة وقهوة ادمي),1,9.4,055 543 1422,SA,Riyadh Province,Riyadh Province,Coffee Shop


In [13]:
app = Flask(__name__)


@app.route('/api' , methods = ['GET'])

def index():
    userInputb = str(request.args['query'])
    userInputa = str(request.args['query'])
    df, data_json = retrieve_places(userInputa , userInputb)

    return data_json


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
